In [1]:
import pandas as pd
from datetime import datetime
import fiscalyear
import xlsxwriter

In [2]:
bySex=True

In [3]:
path = r'\\clsrv111.int.lcog.org\lgs\PSCC\Crime'

In [4]:
data = pd.read_csv(path+'\\Intakes by crime type_Aug 31.csv')

In [5]:
data.head(3)

,Unnamed: 0,SID,sex,Date.of.Admission,Crime.Category,Sentence.Length,Date.Sentence.Begins,Length.of.Stay,Sentence.Number
0,6333,22841186,M,6/1/2018,Driving,24.0,6/1/2018,11.934426,First Sentence
1,6334,18173068,M,6/2/2018,Drug,18.0,6/28/2018,9.770492,First Sentence
2,6335,12094477,M,6/4/2018,Drug,18.0,5/9/2018,10.098361,First Sentence


In [6]:
data.rename(columns = {'Unnamed: 0':'ID'}, inplace = True)

In [7]:
data.columns = list(map(lambda x: x.replace('.', ' '), data.columns))

In [8]:
def get_fiscal_year(datestring):
    date = datetime.strptime(datestring, '%m/%d/%Y').date()
    #initialize the current year
    year_of_date=date.year
    #initialize the current financial year start date
    fiscal_year_start_date = datetime.strptime('7/1/'+str(year_of_date),'%m/%d/%Y').date()
    if date < fiscal_year_start_date:
            return 'July 1, '+ str(fiscal_year_start_date.year-1)+' – June 30, '+ str(fiscal_year_start_date.year)
    else:
            return 'July 1, '+ str(fiscal_year_start_date.year)+' – June 30, '+ str(fiscal_year_start_date.year+1)

In [9]:
data.loc[:, 'Time Period'] = data['Date of Admission'].apply(lambda x: get_fiscal_year(x))

In [10]:
if bySex:
    df = data.groupby(["Time Period", "Crime Category", "Sentence Number", "sex"])['SID'].nunique().reset_index().rename(columns= {'SID':'Count'})
    df.to_excel(path+'\\Crime_Counts_Sex.xlsx', index=False)
    file = path+'\\Crime_Counts_Sex_Formatted.xlsx'
else:
    df = data.groupby(["Time Period", "Crime Category", "Sentence Number"])['SID'].nunique().reset_index().rename(columns= {'SID':'Count'})
    df.to_excel(path+'\\Crime_Counts.xlsx', index=False)
    file = path+'\\Crime_Counts_Formatted.xlsx'

In [11]:
def getStartCells(colnm='Time Period'):
    startCells = [1]
    for row in range(2,len(df)+1):
        if (df.loc[row-1,colnm] != df.loc[row-2,colnm]):
            startCells.append(row)
    return startCells

In [12]:
def mergeFormat(startCells, colnm, colnb, merge_format):
    lastRow = len(df)

    for row in startCells:
        try:
            endRow = startCells[startCells.index(row)+1]-1
            if row == endRow:
                worksheet.write(row, colnb, df.loc[row-1, colnm], merge_format)
            else:
                worksheet.merge_range(row, colnb, endRow, colnb, df.loc[row-1, colnm], merge_format)
        except IndexError:
            if row == lastRow:
                worksheet.write(row, colnb, df.loc[row-1, colnm], merge_format)
            else:
                worksheet.merge_range(row, colnb, lastRow, colnb, df.loc[row-1, colnm], merge_format)

In [13]:
# Create the list where we'll capture the cells that appear for 1st time,
# add the 1st row and we start checking from 2nd row until end of df

writer = pd.ExcelWriter(file, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Counts', index=False)
workbook = writer.book
worksheet = writer.sheets['Counts']
merge_format = workbook.add_format({'align': 'center', 'valign': 'vcenter', 'border': 1})

mergeFormat(startCells=getStartCells(colnm='Time Period'), 
            colnm='Time Period', 
            colnb=0, 
            merge_format=merge_format)

mergeFormat(startCells=getStartCells(colnm='Crime Category'), 
            colnm='Crime Category', 
            colnb=1, 
            merge_format=merge_format)
writer.save()